In [1]:
#%pip install sentence_transformers==3.0.1
#%pip install xformers

In [2]:
# from sentence_transformers import SentenceTransformer

# query_prompt_name = "s2p_query"

# # for each post, per topic, queries is []
# queries = [
#     "What are some ways to reduce stress?",
#     "What are the benefits of drinking green tea?",
# ]
# # 
# docs = [
#     "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
#     "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
# ]

# # ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)
# query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
# doc_embeddings = model.encode(docs)
# print(query_embeddings.shape, doc_embeddings.shape)


# similarities = model.similarity(query_embeddings, doc_embeddings)
# print(similarities)
# # tensor([[0.8179, 0.2958],
# #         [0.3194, 0.7854]])



In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import os


os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
torch.cuda.empty_cache()
print(torch.cuda.is_available())


# Load the pre-trained Sentence Transformer model
# mdl_name = 'stella_en_1.5B_v5' # oom
mdl_name = 'stella_en_400M_v5'
model = SentenceTransformer("dunzhang/"+mdl_name, trust_remote_code=True).cuda()

# Function to generate embeddings
def get_embeddings(text):
    # Generate embeddings
    embedding = model.encode(text)
    return embedding


/home/yg9bq/miniconda3/envs/embed/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


True


Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
folder = "llama_data"
res_df = pd.read_csv("../../"+folder+"/results.csv")
res_df = res_df.fillna("")
fea_df = pd.read_csv("../../data/fea_df.csv")

fea_df['description'] = fea_df['description'].apply(
    lambda x: re.sub(r"Must related to [^(]*\(?.*?\)?\.", "", x).strip()
)

In [5]:
# example

# loop through each sm_id in res_df.sm_id
# loop through each topic in fea_df.fea
sm_id = "104a474"
topic = "bodyhate"


refer_str = fea_df.loc[fea_df.fea==topic, 'description'].tolist()[0]
returned_str = res_df.loc[res_df.sm_id==sm_id, topic+"_phrases"].tolist()[0]
returned_str_ls = returned_str.split(";") # split returned_str by ";"["i wear baggy clothes to hide what i look like."]#
query_prompt_name = "s2p_query"
query_embeddings = model.encode([refer_str], prompt_name=query_prompt_name)
doc_embeddings = model.encode(returned_str_ls)
# print(query_embeddings.shape, doc_embeddings.shape)
cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
res_str = "; ".join([f"{returned_str_ls[i]} ({cos_scores[i]})" for i in range(len(returned_str_ls))]) + ";"

res_str


'i wear baggy clothes to hide what i look like (0.4440997242927551);  people have noticed (0.2869022488594055);'

In [6]:
# from joblib import Parallel, delayed
# import pandas as pd
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # Define the function to process each batch of res_df
# def process_batch(res_df_batch, fea_df):
#     res_df_with_cos_batch = res_df_batch.copy()
#     for sm_id in tqdm(res_df_batch.sm_id.unique(), desc="Processing sm_id"):
#         for topic in fea_df.fea.unique():
#             refer_str = fea_df.loc[fea_df.fea == topic, 'description'].tolist()[0]
#             returned_str = res_df_batch.loc[res_df_batch.sm_id == sm_id, f"{topic}_phrases"].tolist()[0]
            
#             if returned_str:
#                 returned_str_ls = returned_str.split(";")
#                 query_prompt_name = "s2p_query"
#                 query_embeddings = model.encode([refer_str], prompt_name=query_prompt_name)
#                 doc_embeddings = model.encode(returned_str_ls)
#                 cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
#                 res_str = "; ".join([f"{returned_str_ls[i]} ({cos_scores[i]})" for i in range(len(returned_str_ls))]) + ";"
#                 res_df_with_cos_batch.loc[res_df_with_cos_batch.sm_id == sm_id, f"{topic}_phrases"] = res_str
#     return res_df_with_cos_batch

# # # Split res_df into 10 batches
# # res_df_batches = np.array_split(res_df, 1)

# # # Run each batch in parallel using joblib
# # results = Parallel(n_jobs=1)(delayed(process_batch)(batch, fea_df) for batch in res_df_batches)

# # # Combine results back into the main DataFrame
# # res_df_with_cos = pd.concat(results)

# # res_df_with_cos = process_batch(res_df, fea_df)


In [7]:


def process_batch(res_df_batch, fea_df, folder, mdl_name, model):
    res_df_with_cos_batch = res_df_batch.copy()
    sm_id_counter = 0  # Counter to track the number of processed sm_id
    
    for sm_id in tqdm(res_df_batch.sm_id.unique(), desc="Processing sm_id"):
        for topic in fea_df.fea.unique():
            refer_str = fea_df.loc[fea_df.fea == topic, 'description'].tolist()[0]
            returned_str = res_df_batch.loc[res_df_batch.sm_id == sm_id, f"{topic}_phrases"].tolist()[0]
            
            if returned_str:
                returned_str_ls = returned_str.split(";")
                query_prompt_name = "s2p_query"
                query_embeddings = model.encode([refer_str], prompt_name=query_prompt_name)
                doc_embeddings = model.encode(returned_str_ls)
                cos_scores = cosine_similarity(query_embeddings, doc_embeddings)[0]
                res_str = "; ".join([f"{returned_str_ls[i]} ({cos_scores[i]})" for i in range(len(returned_str_ls))]) + ";"
                res_df_with_cos_batch.loc[res_df_with_cos_batch.sm_id == sm_id, f"{topic}_phrases"] = res_str
        
        # Increment the counter and check if 1000 sm_id have been processed
        sm_id_counter += 1
        if sm_id_counter % 1000 == 0:
            # Write intermediate results to CSV
            output_path = os.path.join("../../", folder, f"results_with_cos_{mdl_name}.csv")
            res_df_with_cos_batch.to_csv(output_path, index=False)
            print(f"Intermediate results written to {output_path} at {sm_id_counter} sm_id")
    
    # Final write to ensure all data is saved at the end
    output_path = os.path.join("../../", folder, f"results_with_cos_{mdl_name}.csv")
    res_df_with_cos_batch.to_csv(output_path, index=False)
    print(f"Final results written to {output_path}")

    #return res_df_with_cos_batch


In [ ]:
process_batch(res_df, fea_df, folder, mdl_name, model)

Processing sm_id:   1%|▏         | 1000/77175 [05:52<30:24:11,  1.44s/it]

Intermediate results written to ../../llama_data/results_with_cos_stella_en_400M_v5.csv at 1000 sm_id


Processing sm_id:   2%|▏         | 1691/77175 [10:02<10:22:25,  2.02it/s]

In [26]:
res_df_with_cos.to_csv("../../"+folder+"/results_with_cos_"+mdl_name+".csv", index=False)